In [2]:
# in terminal
'''
mongoimport --type json -d texasSchoolsDB -c scores_finances --drop --jsonArray scores_finances.json
mongoimport --type json -d texasSchoolsDB -c school_info --drop --jsonArray texas-school.json
mongoimport --type json -d texasSchoolsDB -c coordinates --drop --jsonArray finance_coordinates.json

'''

'\nmongoimport --type json -d texasSchoolsDB -c scores_finances --drop --jsonArray scores_finances.json\nmongoimport --type json -d texasSchoolsDB -c school_info --drop --jsonArray texas-school.json\nmongoimport --type json -d texasSchoolsDB -c coordinates --drop --jsonArray finance_coordinates.json\n\n'

In [3]:
from pymongo import MongoClient
import json
import folium
from folium.plugins import MarkerCluster
import pandas as pd

In [4]:
mongo = MongoClient(port=27017)

In [5]:
for dbname in mongo.list_database_names():
    if dbname == "texasSchoolsDB":
        print(f'{dbname} <----------')
    else:
        print(dbname)

admin
autosaurus
classDB
config
epa
fruits_db
garden_db
local
met
petsitly_marketing
test
texasSchoolsDB <----------
travel_db
uk_food


In [6]:
db = mongo["texasSchoolsDB"]

In [7]:
db.list_collection_names()

['coordinates', 'scores_finances', 'school_info']

In [8]:
coordinates = db.coordinates

In [10]:
# Create a Folium map
map = folium.Map(location=[29.7604, -95.3698], zoom_start=6)

# Create a feature group for the markers
marker_group = folium.FeatureGroup(name='Markers')

# Fetch data from MongoDB and create markers
for document in coordinates.find({}):
    latitude = float(document.get('Latitude'))  # Convert to float
    longitude = float(document.get('Longitude'))  # Convert to float
    revenue = float(document.get('Total_Operating_Revenue'))  # Convert to int
    student_count = float(document.get('Student_Count'))  # Convert to int
    district_name = document.get('District_Name')

    # Define marker size based on revenue
    def get_marker_size(revenue):
        if revenue < 5000000:
            return 4
        elif 5000000 <= revenue < 10000000:
            return 8
        elif 10000000 <= revenue < 50000000:
            return 12
        elif 50000000 <= revenue < 100000000:
            return 16
        elif 100000000 <= revenue < 500000000:
            return 20
        else:
            return 24

    # Define marker color based on student count
    def get_marker_color(student_count):
        if student_count >= 2275:
            return 'red'
        elif 1315 <= student_count <= 2274:
            return 'purple'
        elif 545 <= student_count <= 1314:
            return 'blue'
        elif 254 <= student_count <= 544:
            return 'yellow'
        elif 105 <= student_count <= 253:
            return 'orange'
        else:
            return 'green'

    # Create popup text
    popup_text = f"ISD Name: {district_name} <br>Revenue: ${revenue:,}<br>Student Count: {student_count:,}"
    size = get_marker_size(revenue)
    color = get_marker_color(student_count)
    marker = folium.CircleMarker(location=[latitude, longitude],
                                      radius=size,
                                      color=color,
                                      fill=True,
                                      fill_color=color,
                                      fill_opacity=0.5,
                                      popup=popup_text,
                                      clickable=True)

    # Add marker to feature group
    marker.add_to(marker_group)

# Add feature group to the map
marker_group.add_to(map)

# Add layer control to the map
folium.LayerControl().add_to(map)

# Save the map as an HTML file
map.save('map_with_markers_from_mongodb.html')